In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as fn
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader

import pytorch_lightning as ln



In [32]:
class BasicLSTM(ln.LightningModule):

    def __init__(self):
        super().__init__()
        
        mean = torch.tensor(0.0)
        std = torch.tensor(1.0)

        #the forget gate weights and bias 
        #(for f_t = sig(wf1 x_t + wf2 h_{t-1} + b_f))
        self.wf1 = nn.Parameter(torch.normal(mean=mean,std=std),
                                requires_grad=True,
                                )
        self.wf2 = nn.Parameter(torch.normal(mean=mean,std=std),
                                requires_grad=True,
                                )
        self.bf = nn.Parameter(torch.tensor(0.0),
                                requires_grad=True,
                                )
        #the input gate weights and bias
        #(for i_t = sig(wi1 x_t + wi2 h_{t-1} + b_i))
        self.wi1 = nn.Parameter(torch.normal(mean=mean,std=std),
                                requires_grad=True,
                                )
        self.wi2 = nn.Parameter(torch.normal(mean=mean,std=std),
                                requires_grad=True,
                                )
        self.bi = nn.Parameter(torch.tensor(0.0),
                                requires_grad=True,
                                )
        #the output gate weights and bias
        #(for o_t = sig(wo1 x_t + wo2 h_{t-1} + b_o))
        self.wo1 = nn.Parameter(torch.normal(mean=mean,std=std),
                        requires_grad=True,
                        )
        self.wo2 = nn.Parameter(torch.normal(mean=mean,std=std),
                                requires_grad=True,
                                )
        self.bo = nn.Parameter(torch.tensor(0.0),
                                requires_grad=True,
                                )
        #the candidate context weights and bias
        #(for c^'_t = sig(wcc1 x_t + wcc2 h_{t-1} + bc_c))
        self.wcc1 = nn.Parameter(torch.normal(mean=mean,std=std),
                                requires_grad=True,
                                )
        self.wcc2 = nn.Parameter(torch.normal(mean=mean,std=std),
                                requires_grad=True,
                                )
        self.bcc = nn.Parameter(torch.tensor(0.0),
                                requires_grad=True,
                                )

    def unit(self, val_in, long_mem, short_mem):
        '''
        INPUTS:
            val_in - input into this step of the unit x_t

            long_mem - the long term memory at this step

            short_mem - the short term memory at this step
        OUTPUTS:

        
        '''
        f_t = torch.sigmoid((val_in*self.wf1)+(short_mem*self.wf2)+(self.bf))

        i_t = torch.sigmoid((val_in*self.wi1)+(short_mem*self.wi2)+(self.bi))

        cc_t = torch.tanh((val_in*self.wcc1)+(short_mem*self.wcc2)+(self.bcc))

        #update the long term memory (c_t)
        update_long_mem = (f_t*long_mem) + (i_t*cc_t)

        o_t = torch.sigmoid((val_in*self.wo1)+(short_mem*self.wo2)+(self.bo))

        #update the short term memory (h_t)
        update_short_mem = o_t*torch.tanh(update_long_mem)

        return ([update_long_mem, update_short_mem])


    def forward(self,):
        pass

    def optimize(self):
        pass

    def train_step(self):
        pass

In [33]:
model = BasicLSTM()

model.__dict__.keys()

dict_keys(['training', '_parameters', '_buffers', '_non_persistent_buffers_set', '_backward_hooks', '_is_full_backward_hook', '_forward_hooks', '_forward_pre_hooks', '_state_dict_hooks', '_load_state_dict_pre_hooks', '_load_state_dict_post_hooks', '_modules', 'exp_save_path', 'current_epoch', 'global_step', 'loaded_optimizer_states_dict', 'trainer', 'logger', 'use_dp', 'use_ddp', 'use_ddp2', 'use_tpu', 'use_amp', '_dtype', '_device', '_example_input_array'])